In [6]:

import torch
import calc_delta_p
import calc_delta_p_v3
# Example Values: ###  #TODO: delete!!!
T = 100
N = 25000
H_matrix = torch.randn((T, 3, 3)).cuda()
H_matrix[:, 2, 2] = 1
gx_chosen_values = torch.randn((T, N)).cuda()
gy_chosen_values = torch.randn((T, N)).cuda()
Jx_chosen_values = torch.randn((T, N)).cuda()
Jy_chosen_values = torch.randn((T, N)).cuda()
current_level_reference_tensor_zero_mean = (torch.randn(
    (1, N)).cuda()).repeat(T, 1)  # this should be (1 ,N)
current_level_input_tensor_warped = torch.randn((T, N)).cuda()


In [7]:

def images_and_gradients_to_delta_p_yuri(H_matrix,
                                         current_level_reference_tensor_zero_mean,
                                         current_level_input_tensor_warped,
                                         Jx_chosen_values, Jy_chosen_values,
                                         gx_chosen_values, gy_chosen_values):
    ### [H_matrix] = [T,3,3]
    ### [Jx_chosen_values] = [T,N]
    ### [Jy_chosen_values] = [T,N]
    ### [gx_chosen_values] = [T,N]
    ### [gy_chosen_values] = [T,N]
    ### [current_level_reference_tensor_zero_mean] = [T,N]
    ### [current_level_input_tensor_warped] = [T,N]

    ### Correct dimensions for pytorch arithmatic: ###
    Jx_chosen_values = Jx_chosen_values.unsqueeze(
        1).unsqueeze(1)  # -> [T,N,1,1]
    Jy_chosen_values = Jy_chosen_values.unsqueeze(
        1).unsqueeze(1)  # -> [T,N,1,1]
    H_matrix_corrected = H_matrix.unsqueeze(-1)  # -> [T,3,3,1]
    ### Calculate den once: ###

    den = (H_matrix_corrected[:, 2:3, 0:1] * Jx_chosen_values +
           H_matrix_corrected[:, 2:3, 1:2] * Jy_chosen_values +
           H_matrix_corrected[:, 2:3, 2:3])
    denom_inverse = 1 / den

    ### H Transform xy_prime values: ###
    xy_prime_reshaped_X = (H_matrix_corrected[:, 0:1, 0:1] * Jx_chosen_values +
                           H_matrix_corrected[:, 0:1, 1:2] * Jy_chosen_values +
                           H_matrix_corrected[:, 0:1, 2:3]) * denom_inverse
    xy_prime_reshaped_Y = (H_matrix_corrected[:, 1:2, 0:1] * Jx_chosen_values +
                           H_matrix_corrected[:, 1:2, 1:2] * Jy_chosen_values +
                           H_matrix_corrected[:, 1:2, 2:3]) * denom_inverse

    ### Correct Jx,Jy values: ###
    Jx_chosen_values = Jx_chosen_values * denom_inverse  # element-wise
    Jy_chosen_values = Jy_chosen_values * denom_inverse  # element-wise

    #### Get final Jxx,Jxy,Jyy,Jyx values: ####
    Jxx_prime = Jx_chosen_values * xy_prime_reshaped_X  # element-wise.
    Jyx_prime = Jy_chosen_values * xy_prime_reshaped_X
    Jxy_prime = Jx_chosen_values * xy_prime_reshaped_Y  # element-wise
    Jyy_prime = Jy_chosen_values * xy_prime_reshaped_Y

    # ### Get final jacobian of the H_matrix with respect to the different parameters: ###
    # J_list = [Jx_chosen_values, Jy_chosen_values, Jxx_prime, Jxy_prime, Jyx_prime, Jyy_prime]

    ### Yuri calculations: ###
    current_level_reference_tensor_zero_mean = current_level_reference_tensor_zero_mean.unsqueeze(
        1).unsqueeze(1)  # ->[T,1,1,N]
    current_level_input_tensor_warped = current_level_input_tensor_warped.unsqueeze(
        1).unsqueeze(1)  # ->[T,1,1,N]
    gx_chosen_values = gx_chosen_values.unsqueeze(
        1).unsqueeze(1)  # ->[T,1,1,N]
    gy_chosen_values = gy_chosen_values.unsqueeze(
        1).unsqueeze(1)  # ->[T,1,1,N]

    delta_p = calc_delta_p.ecc_calc_delta_p(gx_chosen_values, gy_chosen_values,
                                            Jx_chosen_values, Jy_chosen_values,
                                            Jxx_prime, Jxy_prime, Jyx_prime, Jyy_prime,
                                            current_level_reference_tensor_zero_mean,
                                            current_level_input_tensor_warped)
    return delta_p



In [8]:
G1, Gt1, Gw1, C1, delta_p1 = images_and_gradients_to_delta_p_yuri(H_matrix,
                                                current_level_reference_tensor_zero_mean,
                                                current_level_input_tensor_warped,
                                                Jx_chosen_values, Jy_chosen_values,
                                                gx_chosen_values, gy_chosen_values)

In [9]:
G2, Gt2, Gw2, C2, delta_p2 = calc_delta_p_v3.ecc_calc_delta_p(H_matrix,
                                            current_level_reference_tensor_zero_mean,
                                            current_level_input_tensor_warped,
                                            Jx_chosen_values, Jy_chosen_values,
                                            gx_chosen_values, gy_chosen_values)

In [35]:
# print ( (0.5*(delta_p2-delta_p1)/ (delta_p2.abs()+delta_p1.abs())).abs().argmax(dim=0))
print ( (0.5*(delta_p2-delta_p1)/ (delta_p2.abs()+delta_p1.abs())).abs().max(dim=0))

torch.return_types.max(
values=tensor([[0.0204],
        [0.1582],
        [0.2224],
        [0.2361],
        [0.0560],
        [0.2480],
        [0.0074],
        [0.0165]], device='cuda:0'),
indices=tensor([[15],
        [15],
        [15],
        [15],
        [15],
        [15],
        [15],
        [15]], device='cuda:0'))


In [36]:
print ( (0.5*(delta_p2-delta_p1)/ (delta_p2.abs()+delta_p1.abs())).abs()[15])

tensor([[0.0204],
        [0.1582],
        [0.2224],
        [0.2361],
        [0.0560],
        [0.2480],
        [0.0074],
        [0.0165]], device='cuda:0')


In [20]:
print(delta_p1[15])
print(delta_p2[15])

tensor([[ 1.0931e-01],
        [-4.0909e-02],
        [-1.8496e-06],
        [-5.9156e-03],
        [-9.4704e-02],
        [ 1.0142e-06],
        [ 1.2064e+00],
        [-8.9188e-01]], device='cuda:0')
tensor([[ 1.1861e-01],
        [-7.8766e-02],
        [-4.8131e-06],
        [-2.1210e-03],
        [-7.5638e-02],
        [ 3.0100e-06],
        [ 1.1711e+00],
        [-8.3484e-01]], device='cuda:0')


In [29]:
print((G1[15]-G2[15]).argmax(dim=0))

tensor([11684, 16942, 12902, 11684, 16942, 12902,     0,     0],
       device='cuda:0')


In [33]:
print((G1[15,11684]-G2[15,11684]))
# print(G1[15,11684])
# print(G2[15,11684])

tensor([ 4.1687e-02, -3.4576e-02,  1.7700e+02,  2.8290e-02, -2.3468e-02,
         1.2000e+02,  0.0000e+00,  0.0000e+00], device='cuda:0')


In [43]:
print(G1.max())


tensor(3.7431e+11, device='cuda:0')


In [50]:
print((G1[15]-G2[15]))
print(())

tensor(80907.1250, device='cuda:0')
()
